In [9]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from math import prod
import numpy as np
from sklearn.feature_selection import RFECV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt


## Validação dos dados EOG

Neste notebook está incluído os seguintes passos:
- Aplicação de características;
- Criação do vetor de características;
- Normalização de dados;
- Seleção de características;
- Classificação dos dados.

Uma característica é uma propriedade individual mensurável ou característica de um fenômeno que está sendo observado. Em nosso caso de EOG, uma característica pode ser extraída no domínio do tempo ou no domínio da frequência. As características a seguir foram retiradas do artigo *EMG Feature Extraction for Tolerance of White Gaussian Noise* \[1\].

#### Domínio do tempo

1. Willison Amplitude (WAMP)

    > $ \sum_{i=1}^{N-1}f(|x_i - x_{i+1}|) $
    
    > $ f(x) = \begin{cases} 1 & \text{if } x \gt threshold \\ 0 & \text{otherwise} \end{cases} $

2. Variance of EMG (VAR)

    > $ \frac{1}{N-1}\sum_{i=1}^{N}x_i^2 $

3. Root Mean Square (RMS)

    > $ \sqrt{\frac{1}{N}\sum_{i=1}^{N}|x_i|^2} $

4. Waveform Length (WL)
    
    > $ \sum_{i=1}^{N-1}|x_{i+1} - x_i| $

5. Zero Crossing (ZC)

    > $ \sum_{i=1}^{N}sgn(x_i) $
    
    > $ sgn(x) = \begin{cases} 1 & \text{if } x_i * x_{i+1} \leq 0 \\ 0 & \text{otherwise} \end{cases} $

#### Domínio da frequência

1. Median Frequency (FMD)

    > $ \frac{1}{2}\sum_{j=1}^{M}PSD_j $

2. Mean Frequency (FMN)

    > $\sum_{j=1}^{M}f_j PSD_j / \sum_{j=1}^{M}PSD_j$
    
    > $ f_j = j * SampleRate / 2 * M $

3. Modified Median Frequency (MMDF)

    > $ \frac{1}{2}\sum_{j=1}^{M}A_j $
    
    > $ A_j = Amplitude\ do\ espectro\ j $

4. Modified Frequency Mean (MMNF)

    > $ \sum_{j=1}^{M}f_jAj / \sum_{j=1}^{M}Aj $


\[1\] Phinyomark, Angkoon & Limsakul, Chusak & Phukpattaranont, P.. (2008). EMG Feature Extraction for Tolerance of White Gaussian Noise.
[Disponível neste link](https://www.researchgate.net/publication/263765853_EMG_Feature_Extraction_for_Tolerance_of_White_Gaussian_Noise)

**Tarefa 1**: Descrever as características de acordo com o artigo citado e outros disponíveis relacionados. O que está querendo "ser visto" em cada característica? Qual é o significado matemático de cada uma delas?

#### Aplicando as características

É necessário implementar as características, geralmente em formato de funções ou métodos, para que seja possível aplicar tais funções aos dados de entrada e obter as características resultantes. A seguir temos a implementação das características `VAR` & `RMS` (domínio do tempo) e `FDM` & `MMDF` (domínio da frequência).

In [10]:
# funções auxiliares
def PSD(w):
    ''' definição da função PSD para o sinal no domínio da frequência '''
    return np.abs(w) ** 2

def sign(x):
    return x[:-1] * x[1:]

# funções de extração de características
def var(x):
    return np.sum(x ** 2, axis=-1) / (np.prod(x.shape) - 1)

def rms(x):
    return np.sqrt(np.sum(np.abs(x) ** 2, axis=-1) / (np.prod(x.shape) - 1))

def fmd(w):
    return np.sum(PSD(w), axis=-1) / 2

def mmdf(w):
    return np.sum(np.abs(w), axis=-1) / 2

def wl(x):
    return np.sum(np.abs(np.diff(x)), axis=-1)

def zc(x):
    return np.sum(sign(x) <= 0, axis=-1)

def wamp(x, threshold):
    differences = np.diff(x)
    crossings = np.where(np.abs(differences) > threshold, 1, 0)
    return np.sum(crossings)

def fmn(x):
    return np.sum(PSD(x))

def mmnf(x):
    return np.sum(PSD(x)) / 2

**Tarefa 2**: Implemente todas as características apresentadas neste tutorial em formato de funções. Sinta-se livre também para buscar e implementar características além das apresentadas, citando as fontes de tais características.


#### Vetor de características

Ao final da implementação e seleção das características, deve ser escolhida as características e então teremos um vetor com todas elas implementadas.

O vetor de características estará organizado da seguinte forma (exemplo p/ VAR, RMS, RDM e MMDF):

| ID sample | VAR1 | RMS1 | FMD1 | MMDF1 | VAR2 | RMS2 | FMD2 | MMDF2 | Classe |
|:---------:|:----:|:----:|:----:|:-----:|------|------|------|-------|:------:|
|     1     |  v1  |  v1  |  v1  |   v1  | v1   | v1   | v1   | v1    |    0   |
|     2     |  v2  |  v2  |  v2  |   v2  | v2   | v2   | v2   | v2    |    0   |
|    ...    |  ... |  ... |  ... |  ...  | ...  | ...  | ...  | ...   |   ...  |
|     N     |  vN  |  vN  |  vN  |   vN  | vN   | vN   | vN   | vN    |    7   |

#### Implementação do vetor

In [11]:
# carregando dados do "prepare.ipynb"

todos_time = np.load("datasets/todos_time.npy")
todos_frequencia = np.load("datasets/todos_frequencia.npy")

gabi_time = np.load("datasets/gabi_time.npy")
gabi_frequencia = np.load("datasets/gabi_frequencia.npy")

jesse_time = np.load("datasets/jesse_time.npy")
jesse_frequencia = np.load("datasets/jesse_frequencia.npy")

### Todos participantes

In [4]:
print("Shape dos vetores originais:", todos_time.shape, todos_frequencia.shape)

# mudando o de todos para 56, 33, 2, 64 e 56, 33, 2, 33
todos_time = todos_time.reshape((-1, 33, 2, 64))
todos_frequencia = todos_frequencia.reshape((-1, 33, 2, 33))

print("Shape dos vetores mudança 1:", todos_time.shape, todos_frequencia.shape)


# aplicando características
data_var = var(todos_time)
data_rms = rms(todos_time)
data_wl = wl(todos_time)
data_zc = zc(todos_time)

time_median = np.median(todos_time)
data_wamp = wamp(todos_time, time_median) # threshold ser a mediana dos valores totais

data_fmd = fmd(todos_frequencia)
data_fmn = fmn(todos_frequencia)
data_mmdf = mmdf(todos_frequencia)
data_mmnf = mmnf(todos_frequencia)

data_wamp.shape, data_var.shape, data_rms.shape, data_wl.shape, data_zc.shape, data_fmd.shape, data_fmn.shape, data_mmdf.shape, data_mmnf.shape

Shape dos vetores originais: (2, 28, 33, 2, 64) (2, 28, 33, 2, 33)
Shape dos vetores mudança 1: (56, 33, 2, 64) (56, 33, 2, 33)


((),
 (56, 33, 2),
 (56, 33, 2),
 (56, 33, 2),
 (55, 33, 2),
 (56, 33, 2),
 (),
 (56, 33, 2),
 ())

In [5]:
# União do vetor de características inicial
features = np.array([data_var, data_rms, data_wl, data_fmd, data_mmdf])
print(features.shape)

# organização das dimensões
features = features.transpose(2, 1, 0, 3)
print(features.shape)

# criar vetor de características definitivo
features = features.reshape(features.shape[0] * features.shape[1],
                            features.shape[2] * features.shape[3])

features.shape

(5, 56, 33, 2)
(33, 56, 5, 2)


(1848, 10)

In [6]:
# aplicando normalização
X_normalizado = StandardScaler().fit_transform(features)

# código

In [7]:
# obtendo labels

labels_str = ['dir', 'esq', 'cima', 'baixo', 'cima', 'baixo',
'baixo', 'esq', 'dir', 'baixo', 'dir', 'dir', 'esq', 'cima',
'baixo', 'cima', 'esq', 'dir', 'cima', 'esq', 'baixo', 'esq',
'dir', 'esq', 'cima', 'dir', 'cima', 'baixo']

# transformando para numérico
lab_dict = {'dir': 0, 'esq': 1, 'cima': 2, 'baixo': 3}
labels_num = [lab_dict[item] for item in labels_str]

print(labels_num)

# criação do vetor de labels final
y = np.repeat(labels_num, int(features.shape[0] / len(labels_num)))
y.shape

[0, 1, 2, 3, 2, 3, 3, 1, 0, 3, 0, 0, 1, 2, 3, 2, 1, 0, 2, 1, 3, 1, 0, 1, 2, 0, 2, 3]


(1848,)

In [8]:
# aplicando seleção de características
X_normalizado.shape, y.shape
# código

((1848, 10), (1848,))

In [18]:
# aplicando seleção de características
rfe = RFECV(SVC(kernel="linear"), step=0.000001, min_features_to_select=1, cv=10)
X_final = rfe.fit_transform(X_normalizado, y)
print(X_final.shape)

(1848, 10)


In [19]:
# aplicando a classificação

x_train, x_test, y_train, y_test = train_test_split(X_final, y, test_size=0.003, random_state=42)

clf = SVC(kernel='linear', C=1, random_state=42, probability=True)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)  

print("accuracy svm", accuracy_score(y_test, y_pred))
print("f1_score svm", f1_score(y_test, y_pred, average="weighted"))

# Criando e treinando o modelo RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(x_train, y_train)

# Fazendo previsões no conjunto de teste
y_pred = rf_model.predict(x_test)

print("accuracy random forest", accuracy_score(y_test, y_pred))
print("f1_score random forest", f1_score(y_test, y_pred, average="weighted"))


accuracy svm 0.8333333333333334
f1_score svm 0.8222222222222223
accuracy random forest 0.3333333333333333
f1_score random forest 0.3333333333333333


In [32]:
rfe = RFECV(RandomForestClassifier(n_estimators=100, random_state=42), step=0.000001, min_features_to_select=1, cv=3, scoring='accuracy')
X_final = rfe.fit_transform(X_normalizado, y)
print(X_final.shape)


(1848, 6)


In [40]:
# aplicando a classificação

x_train, x_test, y_train, y_test = train_test_split(X_final, y, test_size=0.3, random_state=42, stratify=y)

clf = SVC(kernel='linear', C=1, random_state=42, probability=True)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)  

print("accuracy svm", accuracy_score(y_test, y_pred))
print("f1_score svm", f1_score(y_test, y_pred, average="weighted"))

# Criando e treinando o modelo RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(x_train, y_train)

# Fazendo previsões no conjunto de teste
y_pred = rf_model.predict(x_test)

print("accuracy random forest", accuracy_score(y_test, y_pred))
print("f1_score random forest", f1_score(y_test, y_pred, average="weighted"))

accuracy svm 0.3207207207207207
f1_score svm 0.292411109157848
accuracy random forest 0.42342342342342343
f1_score random forest 0.4266260128542383


### Participante Jesse

In [12]:
print("Shape dos vetores originais:", todos_time.shape, todos_frequencia.shape)

# mudando o de todos para 56, 33, 2, 64 e 56, 33, 2, 33
jesse_time = jesse_time.reshape((-1, 33, 2, 64))
jesse_frequencia = jesse_frequencia.reshape((-1, 33, 2, 33))

print("Shape dos vetores mudança 1:", todos_time.shape, todos_frequencia.shape)


# aplicando características
data_var = var(jesse_time)
data_rms = rms(jesse_time)
data_wl = wl(jesse_time)
data_zc = zc(jesse_time)

time_median = np.median(jesse_time)
data_wamp = wamp(jesse_time, time_median) # threshold ser a mediana dos valores totais

data_fmd = fmd(jesse_frequencia)
data_fmn = fmn(jesse_frequencia)
data_mmdf = mmdf(jesse_frequencia)
data_mmnf = mmnf(jesse_frequencia)

data_wamp.shape, data_var.shape, data_rms.shape, data_wl.shape, data_zc.shape, data_fmd.shape, data_fmn.shape, data_mmdf.shape, data_mmnf.shape

Shape dos vetores originais: (2, 28, 33, 2, 64) (2, 28, 33, 2, 33)
Shape dos vetores mudança 1: (2, 28, 33, 2, 64) (2, 28, 33, 2, 33)


((),
 (28, 33, 2),
 (28, 33, 2),
 (28, 33, 2),
 (27, 33, 2),
 (28, 33, 2),
 (),
 (28, 33, 2),
 ())

In [13]:
# União do vetor de características inicial
features = np.array([data_var, data_rms, data_wl, data_fmd, data_mmdf])
print(features.shape)

# organização das dimensões
features = features.transpose(2, 1, 0, 3)
print(features.shape)

# criar vetor de características definitivo
features = features.reshape(features.shape[0] * features.shape[1],
                            features.shape[2] * features.shape[3])

features.shape

(5, 28, 33, 2)
(33, 28, 5, 2)


(924, 10)

In [14]:
# aplicando normalização
X_normalizado = StandardScaler().fit_transform(features)

# código

In [15]:
# obtendo labels

labels_str = ['dir', 'esq', 'cima', 'baixo', 'cima', 'baixo',
'baixo', 'esq', 'dir', 'baixo', 'dir', 'dir', 'esq', 'cima',
'baixo', 'cima', 'esq', 'dir', 'cima', 'esq', 'baixo', 'esq',
'dir', 'esq', 'cima', 'dir', 'cima', 'baixo']

# transformando para numérico
lab_dict = {'dir': 0, 'esq': 1, 'cima': 2, 'baixo': 3}
labels_num = [lab_dict[item] for item in labels_str]

print(labels_num)

# criação do vetor de labels final
y = np.repeat(labels_num, int(features.shape[0] / len(labels_num)))
y.shape

[0, 1, 2, 3, 2, 3, 3, 1, 0, 3, 0, 0, 1, 2, 3, 2, 1, 0, 2, 1, 3, 1, 0, 1, 2, 0, 2, 3]


(924,)

In [16]:
# aplicando seleção de características
X_normalizado.shape, y.shape
# código

((924, 10), (924,))

In [101]:
# aplicando seleção de características
rfe = RFECV(SVC(kernel="linear"), step=0.000001, min_features_to_select=1, cv=5)
X_final = rfe.fit_transform(X_normalizado, y)
print(X_final.shape)

(924, 6)


In [102]:
# aplicando a classificação

x_train, x_test, y_train, y_test = train_test_split(X_final, y, test_size=0.3, random_state=42, stratify=y)

clf = SVC(kernel='linear', C=1, random_state=42, probability=True)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)  

print("accuracy svm", accuracy_score(y_test, y_pred))
print("f1_score svm", f1_score(y_test, y_pred, average="weighted"))

# Criando e treinando o modelo RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(x_train, y_train)

# Fazendo previsões no conjunto de teste
y_pred = rf_model.predict(x_test)

print("accuracy random forest", accuracy_score(y_test, y_pred))
print("f1_score random forest", f1_score(y_test, y_pred, average="weighted"))

accuracy svm 0.35251798561151076
f1_score svm 0.31506707284189284
accuracy random forest 0.381294964028777
f1_score random forest 0.38414743956733766


In [103]:
rfe = RFECV(RandomForestClassifier(n_estimators=100, random_state=42), step=0.000001, min_features_to_select=1, cv=10, scoring='accuracy')
X_final = rfe.fit_transform(X_normalizado, y)
print(X_final.shape)


(924, 10)


In [104]:
# aplicando a classificação

x_train, x_test, y_train, y_test = train_test_split(X_final, y, test_size=0.3, random_state=42, stratify=y)

clf = SVC(kernel='linear', C=1, random_state=42, probability=True)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)  

print("accuracy svm", accuracy_score(y_test, y_pred))
print("f1_score svm", f1_score(y_test, y_pred, average="weighted"))

# Criando e treinando o modelo RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(x_train, y_train)

# Fazendo previsões no conjunto de teste
y_pred = rf_model.predict(x_test)

print("accuracy random forest", accuracy_score(y_test, y_pred))
print("f1_score random forest", f1_score(y_test, y_pred, average="weighted"))

accuracy svm 0.35251798561151076
f1_score svm 0.3230395503776799
accuracy random forest 0.44244604316546765
f1_score random forest 0.44436532705445353


### Participante Gabi

In [41]:
print("Shape dos vetores originais:", todos_time.shape, todos_frequencia.shape)

# mudando o de todos para 56, 33, 2, 64 e 56, 33, 2, 33
gabi_time = gabi_time.reshape((-1, 33, 2, 64))
gabi_frequencia = gabi_frequencia.reshape((-1, 33, 2, 33))

print("Shape dos vetores mudança 1:", todos_time.shape, todos_frequencia.shape)


# aplicando características
data_var = var(gabi_time)
data_rms = rms(gabi_time)
data_wl = wl(gabi_time)
data_zc = zc(gabi_time)

time_median = np.median(gabi_time)
data_wamp = wamp(gabi_time, time_median) # threshold ser a mediana dos valores totais

data_fmd = fmd(gabi_frequencia)
data_fmn = fmn(gabi_frequencia)
data_mmdf = mmdf(gabi_frequencia)
data_mmnf = mmnf(gabi_frequencia)

data_wamp.shape, data_var.shape, data_rms.shape, data_wl.shape, data_zc.shape, data_fmd.shape, data_fmn.shape, data_mmdf.shape, data_mmnf.shape

Shape dos vetores originais: (56, 33, 2, 64) (56, 33, 2, 33)
Shape dos vetores mudança 1: (56, 33, 2, 64) (56, 33, 2, 33)


((),
 (28, 33, 2),
 (28, 33, 2),
 (28, 33, 2),
 (27, 33, 2),
 (28, 33, 2),
 (),
 (28, 33, 2),
 ())

In [42]:
# União do vetor de características inicial
features = np.array([data_var, data_rms, data_wl, data_fmd, data_mmdf])
print(features.shape)

# organização das dimensões
features = features.transpose(2, 1, 0, 3)
print(features.shape)

# criar vetor de características definitivo
features = features.reshape(features.shape[0] * features.shape[1],
                            features.shape[2] * features.shape[3])

features.shape

(5, 28, 33, 2)
(33, 28, 5, 2)


(924, 10)

*Tarefa 3*: Realização da normalização dos dados utilizando ferramentas já conhecidas

In [43]:
# aplicando normalização
X_normalizado = StandardScaler().fit_transform(features)

# código

*Tarefa 4*: Realização da seleção de características, utilizando ferramentas já conhecidas

In [44]:
# obtendo labels

labels_str = ['dir', 'esq', 'cima', 'baixo', 'cima', 'baixo',
'baixo', 'esq', 'dir', 'baixo', 'dir', 'dir', 'esq', 'cima',
'baixo', 'cima', 'esq', 'dir', 'cima', 'esq', 'baixo', 'esq',
'dir', 'esq', 'cima', 'dir', 'cima', 'baixo']

# transformando para numérico
lab_dict = {'dir': 0, 'esq': 1, 'cima': 2, 'baixo': 3}
labels_num = [lab_dict[item] for item in labels_str]

print(labels_num)

# criação do vetor de labels final
y = np.repeat(labels_num, int(features.shape[0] / len(labels_num)))
y.shape

[0, 1, 2, 3, 2, 3, 3, 1, 0, 3, 0, 0, 1, 2, 3, 2, 1, 0, 2, 1, 3, 1, 0, 1, 2, 0, 2, 3]


(924,)

In [45]:
# aplicando seleção de características
X_normalizado.shape, y.shape
# código

((924, 10), (924,))

In [75]:
# aplicando seleção de características
rfe = RFECV(SVC(kernel="linear"), step=0.000001, min_features_to_select=1, cv=6)
X_final = rfe.fit_transform(X_normalizado, y)
print(X_final.shape)

(924, 8)


Criação do vetor de *labels*

*Tarefa 5*: Realização da classificação utilizando `SVM`.

In [76]:
# aplicando a classificação

x_train, x_test, y_train, y_test = train_test_split(X_final, y, test_size=0.3, random_state=42, stratify=y)

clf = SVC(kernel='linear', C=1, random_state=42, probability=True)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)  

print("accuracy svm", accuracy_score(y_test, y_pred))
print("f1_score svm", f1_score(y_test, y_pred, average="weighted"))

# Criando e treinando o modelo RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(x_train, y_train)

# Fazendo previsões no conjunto de teste
y_pred = rf_model.predict(x_test)

print("accuracy random forest", accuracy_score(y_test, y_pred))
print("f1_score random forest", f1_score(y_test, y_pred, average="weighted"))

accuracy svm 0.39568345323741005
f1_score svm 0.3908264680203217
accuracy random forest 0.381294964028777
f1_score random forest 0.38405853182642513


In [83]:
rfe = RFECV(RandomForestClassifier(n_estimators=100, random_state=42), step=0.000001, min_features_to_select=1, cv=10, scoring='accuracy')
X_final = rfe.fit_transform(X_normalizado, y)
print(X_final.shape)


(924, 1)


In [84]:
# aplicando a classificação

x_train, x_test, y_train, y_test = train_test_split(X_final, y, test_size=0.3, random_state=42, stratify=y)

clf = SVC(kernel='linear', C=1, random_state=42, probability=True)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)  

print("accuracy svm", accuracy_score(y_test, y_pred))
print("f1_score svm", f1_score(y_test, y_pred, average="weighted"))

# Criando e treinando o modelo RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(x_train, y_train)

# Fazendo previsões no conjunto de teste
y_pred = rf_model.predict(x_test)

print("accuracy random forest", accuracy_score(y_test, y_pred))
print("f1_score random forest", f1_score(y_test, y_pred, average="weighted"))

accuracy svm 0.2697841726618705
f1_score svm 0.16522545257104776
accuracy random forest 0.381294964028777
f1_score random forest 0.3802140966558557
